In [1]:
import sys
# sys.path.append('pytorchvideo')
from pytorchvideo.data import LabeledVideoDataset

ModuleNotFoundError: No module named 'pytorchvideo'

In [2]:
from glob import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Data Augmentation Process

In [3]:
# Augmentation process
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from torch.utils.data import DataLoader

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,

    UniformTemporalSubsample,
    Permute
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

/opt/conda/lib/python3.9/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [4]:
video_transform = Compose([
    ApplyTransformToKey(key='video',
        transform = Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x:x/255),
        NormalizeVideo(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
        RandomShortSideScale(min_size=256, max_size=256),
        CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5)
    ]),
    ),
])

# Train Data Loader

In [5]:
# Labeled video dataset
train_dataset = labeled_video_dataset('../data_temporal/train_root',
                                      clip_sampler=make_clip_sampler('random', 2),
                                        transform = video_transform, decode_audio=False)


train_loader = DataLoader(train_dataset, batch_size=5, num_workers=0, pin_memory=True)

In [6]:
# train_batch = next(iter(train_loader))

In [7]:
# train_batch.keys()

In [8]:
# train_batch['video'].shape, train_batch['label'].shape


In [9]:
# Validation Data Loader

In [10]:
# val_dataset = labeled_video_dataset('../data_temporal/val_root',
#                                       clip_sampler=make_clip_sampler('random', 2),
#                                         transform = video_transform, decode_audio=False)


# val_loader = DataLoader(train_dataset, batch_size=5, num_workers=0, pin_memory=True)

In [11]:
# val_batch = next(iter(val_loader))

In [12]:
# val_batch.keys()

In [13]:
# val_batch['video'].shape, val_batch['label'].shape

# Model Architecture

In [14]:
import torch.nn as nn
import torch
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

In [15]:
video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [32]:
# model.py
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel, self).__init__()
        # Model Architecture
        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, 12)
        
        self.lr = 1e-3
        
        self.batch_size = 8
        self.numworker=4 # If decrease it will be slower computation time
        
        # Evaluation Metric
        self.metric = torchmetrics.Accuracy(task='multiclass',   num_classes=12)
        
        # Loss Function
        # self.criterion = nn.BCEWithLogitsLoss()
        self.criterion = nn.CrossEntropyLoss()
        

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params = self.parameters(), lr = self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}


    def train_dataloader(self):
        dataset = labeled_video_dataset('../data_temporal/train_root',
                                      clip_sampler=make_clip_sampler('random', 2),
                                        transform = video_transform, decode_audio=False)


        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
        return loader
    
    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video) # or self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        return {'loss': loss, 'metric': metric.detach()}
    
    
    def on_train_epoch_end(slef, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('training_loss', loss)
        self.log('trainng_metric', metric)
    
    
    def val_dataloader(self):
        dataset = labeled_video_dataset('../data_temporal/val_root',
                                      clip_sampler=make_clip_sampler('random', 2),
                                        transform = video_transform, decode_audio=False)


        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
        return loader
    
    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video) # or self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        return {'loss': loss, 'metric': metric.detach()}
    
    def on_vaidation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)
        
    def test_dataloader(self):
        dataset = labeled_video_dataset('../data_temporal/val_root',
                                      clip_sampler=make_clip_sampler('random', 2),
                                        transform = video_transform, decode_audio=False)


        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworker, pin_memory=True)
        return loader
    
    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self(video) # or self.forward(video)
        return {'label': label.detach(), 'pred': out.detach()}
    
    def on_test_epoch_end(self, outputs):
        label = torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        pred = np.where(pred>0.5,1,0)
        print(classification_report(label, pred))
        
    
    
    
        
        

In [33]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath = 'checkpoints',
                                      filename='file', save_last=True)

lr_monitor = LearningRateMonitor(logging_interval='epoch')

# 10 (total), 5(no improved), 7(interupted), 7(resume)

In [34]:
model = OurModel()
seed_everything(0)

trainer = Trainer(max_epochs = 1,
                  accelerator = 'cpu',
                      precision = 16,
                  accumulate_grad_batches = 2,
                  enable_progress_bar = False,
                  num_sanity_val_steps = 0,
                  callbacks = [lr_monitor, checkpoint_callback],
                  # limit_train_batches = 5, # If you want to limit it 
                  # limit_val_batches = 1, # If you want to limit it 
                  
                  
                 )
                  
# Next to accelerator I removed this:  , devices = -1, because running on cpu.

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model)


  | Name        | Type               | Params
---------------------------------------------------
0 | video_model | EfficientX3d       | 3.8 M 
1 | relu        | ReLU               | 0     
2 | linear      | Linear             | 4.8 K 
3 | metric      | MulticlassAccuracy | 0     
4 | criterion   | CrossEntropyLoss   | 0     
---------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.197    Total estimated model params size (MB)
